In [3]:
from utils_tokens import TOKEN_FILE_PATH 
import os
import pickle
from IPython.display import display, Javascript, HTML
import time
print(TOKEN_FILE_PATH)
def _save_and_set_token_from_js(token_value: str):
    """
    Internal helper function called by JavaScript to save and set the token.
    Do not call this directly from Python.
    """
   
    token_api = token_value
    print(token_value)
    try:
        with open(TOKEN_FILE_PATH, "wb") as f:
            pickle.dump(token_api, f)
        print(f"Token received and saved to '{TOKEN_FILE_PATH}'.")
    except Exception as e:
        print(f"Error saving token to '{TOKEN_FILE_PATH}': {e}")


# --- Main function to check/prompt for token ---
def prompt_for_api_token():
    """
    Checks for a pickled API token. If it exists, loads it.
    Otherwise, prompts the user via JavaScript for the token and pickles it.
    The token will be available in the global 'token_api' variable once set.
    """

    if not os.path.exists(TOKEN_FILE_PATH):
        # File does not exist, prompt for token
        print(f"'{TOKEN_FILE_PATH}' not found. Prompting for API token...")

        js_code = f"""
        var user_token = prompt("Please enter your API token:");
        if (user_token !== null && user_token.trim() !== "") {{
            // Call the Python function `_save_and_set_token_from_js`
            // We need to carefully escape quotes if the token contains them.
            // For simplicity, this assumes no complex quotes in the token.
            console.log("You entered token: " + user_token);
            var escaped_token = user_token.replace(/'/g, "\\'"); // Basic escaping for single quotes
            IPython.notebook.kernel.execute("_save_and_set_token_from_js('" + escaped_token + "')");
        }} else {{
            console.log("API token input cancelled or empty. Token will not be set.");
            IPython.notebook.kernel.execute("print('API token input cancelled or empty. Token not set.')");
        }}
        """
        display(Javascript(js_code))
        # Provide visual feedback to the user that a prompt should appear
        display(HTML("<p><em>A browser pop-up prompt should appear. Please enter your token there.</em></p>"))


# --- Register the helper function for JavaScript to call ---
# This makes _save_and_set_token_from_js callable by IPython.notebook.kernel.execute
# from Javascript.
from IPython.core.interactiveshell import InteractiveShell
# Check if running in IPython (Jupyter)
if 'IPython' in globals():
    InteractiveShell.instance().user_ns['_save_and_set_token_from_js'] = _save_and_set_token_from_js

prompt_for_api_token()

nqch_API_token.pickle
'nqch_API_token.pickle' not found. Prompting for API token...


<IPython.core.display.Javascript object>

In [4]:
import utils_tokens
import time
time.sleep(3)
token = utils_tokens.load_api_token()


API token loaded from 'nqch_API_token.pickle'.


In [5]:
import qibo_client

# authenticate to server through the client instance
client = qibo_client.Client(token)

# run the circuit
device = "sinq20"
project = "personal"

In [6]:
from qibo import Circuit, gates
import numpy as np
circuit = Circuit(20)
#Currently the CZs are calibrated for 3-8, 8-13 and 13-17
q0 = 3
q1 = 8
circuit.add(gates.CNOT(q0, q1))
circuit.add(gates.RX(q0, np.pi/2))
circuit.add(gates.CNOT(q0, q1))
circuit.add(gates.M(q0, q1))
job = client.run_circuit(circuit, device=device, project=project, nshots=1)
result = job.result()
print(result)
result.frequencies()

[2025-06-19 21:02:38,813] INFO: Post new circuit on the server
[2025-06-19 21:02:39,731] INFO: Job posted on server with pid 5e852688cdbd4ef691c0f173f404980f
[2025-06-19 21:02:40,350] INFO: Please wait until your job is completed...


[Qibo 0.2.19|INFO|2025-06-19 21:03:10]: Using qibojit (numba) backend on /CPU:0
[2025-06-19 21:03:10,440] INFO: Using qibojit (numba) backend on /CPU:0


Counter({'00': 1})